In [2]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

from matplotlib import pyplot as plt
from tqdm import tqdm
from helpers.prep.normalization import RecipeNormalizer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)



In [4]:
food_normalized_path = Path("./app/data/test/reduced_food.csv")
food_df = pd.read_csv(food_normalized_path)
# food_raw_path = "./app/data/food_reviews/RAW_recipes.csv"
# food_raw_df = pd.read_csv(food_raw_path)

In [6]:
food_df.head()

,name,steps,description,ingredients,ingredients_in_instructions,clean_instructions
0,arriba baked winter squash mexican style,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...","['butter', 'aluminum_foil', 'squash', 'salt', ...",make a choice and proceed with recipedepending...
1,a bit different breakfast pizza,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...","['sausage', 'brown', 'milk', 'cheese', 'baked'...",preheat oven to 425 degree fpress dough into t...
2,all in the kitchen chili,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...","['chopped', 'onion', 'kidney_bean', 'shredded'...",brown ground_beef in large potadd chopped onio...
3,alouette potatoes,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...","['bake', 'water', 'yellow_bell_pepper', 'red',...",place potato in a large pot of lightly salted ...
4,amish tomato ketchup for canning,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...","['ketchup', 'jar']","mix all ingredient boil for 2 1 2 hour, or unt..."
